In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# 모듈 임포트
import cv2
import numpy as np
import time
import io
import base64
from IPython.display import HTML

In [ ]:
# 원본 동영상 Display
video = io.open('/content/gdrive/MyDrive/CV/Object Detection_YOLO/video/video/street.mp4',
               'r+b').read()

encoded = base64.b64encode(video)
HTML(data = '''<video width = '30%' controls>
                <source src = 'data:video/mp4;base64,{0}' type = 'video/mp4'/>
                </video>'''.format(encoded.decode('ascii')))

In [ ]:
# Detection 할 원본 동영상
file_name = '/content/gdrive/MyDrive/CV/Object Detection_YOLO/video/video/street.mp4'
min_confidence = 0.5
output_name = 'street_output_video.mp4'
elapsed_time = 0        # 총 경과시간 초기화

In [ ]:
# Load YOLO
net = cv2.dnn.readNet('/content/gdrive/MyDrive/CV/Object Detection_YOLO/yolov3.weights',
                      '/content/gdrive/MyDrive/CV/Object Detection_YOLO/yolov3.cfg')
classes = []

with open('/content/gdrive/MyDrive/CV/Object Detection_YOLO/coco.names', 'r') as f:
    # 80개의 Object(class)를 구분할 수 있는 Object의 이름을 classes 배열에 넣는다.
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Object 마다 컬러를 하나씩 다르게 지정
colors = np.random.uniform(0, 255, size = (len(classes), 3))

In [ ]:
# detect & Display 함수 정의
def detectAndDisplay(frame):
    start_time = time.time()
    img = cv2.resize(frame, None, fx=0.9, fy=0.9)
    height, width, channels = img.shape

    # YOLOv3의 Detecting model 3가지(320*320, 416*416, 608*608)
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > min_confidence:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])              # boxing 정보 저장
                confidences.append(float(confidence))   # 신뢰도 저장
                class_ids.append(class_id)              # Class id 저장

    # 박스 안에 박스(노이즈)를 하나로 만들어준다.
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]

            # Class 이름, 신뢰도 표시
            label = '{}: {:.2f}'.format(classes[class_ids[i]], confidences[i] * 100)

            print(i, label)
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            cv2.rectangle(img, (x, y - 20), (x + w, y), color, -1)
            cv2.putText(img, label, (x + 5, y - 5), font, 1, (255, 255, 255), 1)

    process_time = time.time() - start_time

    global elapsed_time
    elapsed_time += process_time     # 총 경과시간 누적

    print('= = = A frame took {:.3f} seconds'.format(process_time))

    # video 를 disk에 output 하기 위해 writer 초기화
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*'DIVX')
        writer = cv2.VideoWriter(output_name, fourcc, 30,
                                (img.shape[1], img.shape[0]), True)
    # disk에 frame write
    if writer is not None:
        writer.write(img)

In [ ]:
# 원본 동영상에서 video stream 읽어오기
cap = cv2.VideoCapture(file_name)
writer = None
if not cap.isOpened:
    print('- -(!)Error opening video capture')
    exit(0)

while True:
    ret,frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()

        # close the writer point
        writer.release()
        print('- -(!) No captured frame - - Break!')
        print('elapsed time {:.3f} seconds'.format(elapsed_time))
        break

    detectAndDisplay(frame)